<a href="https://colab.research.google.com/github/Parv-Agarwal/Internship-project/blob/main/Pre_Trained_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import os
import numpy as np
from PIL import Image

In [2]:
opt = {
    'dataset': 'mnist',
    'batchSize': 64,
    'loadSize': 33,
    'fineSize': 32,
    'nz': 100,               # # of dim for Z
    'ngf': 64,               # # of gen filters in first conv layer
    'ndf': 64,               # # of discrim filters in first conv layer
    'nThreads': 4,           # # of data loading threads to use
    'niter': 10000,          # # of iter at starting learning rate
    'lr': 0.0002,            # initial learning rate for adam
    'beta1': 0.5,            # momentum term of adam
    'ntrain': float('inf'),  # # of examples per epoch
    'display': 0,            # display samples while training
    'display_id': 0,         # display window id
    'gpu': 1,                # gpu = 0 is CPU mode. gpu=X is GPU mode on GPU X
    'name': 'Logfiles',
    'noise': 'normal',       # 'uniform' or 'normal'
    'epoch_save_modulo': 1,
    'manual_seed': 4,        # Seed
    'nc': 3,                 # # of channels in input
    'save': 'logs/',         # Directory to save logs
    'data_root': './data',   # Root directory for datasets
    'lamda': 1,              # Lambda value for GRL
    'baseLearningRate': 0.0002,
    'max_epoch': 10000,
    'gamma': 0.001,
    'power': 0.75,
    'max_epoch_grl': 10000,
    'alpha': 10,
    'num_classes': 10,
    'num_epochs': 10,
    'image_size': 32,
}


In [3]:
# Set random seed
import random
random.seed(opt['manual_seed'])
torch.manual_seed(opt['manual_seed'])
torch.set_num_threads(1)

if torch.cuda.is_available() and opt['gpu'] > 0:
    torch.cuda.manual_seed_all(opt['manual_seed'])
    device = torch.device(f'cuda:{opt["gpu"] - 1}')
else:
    device = torch.device('cpu')

print(f"Random Seed: {opt['manual_seed']}")
print(f"Device: {device}")

Random Seed: 4
Device: cuda:0


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize with MNIST mean and std
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4656551.85it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 134888.37it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 246252.51it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2780287.33it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
class PreTrainedClassifier(nn.Module):
    def __init__(self):
        super(PreTrainedClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(28 * 28, 256),  # Input size is 28*28 for MNIST images
            nn.ReLU(True),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 10),  # 10 output classes for digits 0-9
            nn.LogSoftmax(dim=1)  # LogSoftmax for classification
        )

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the input (batch_size, 28*28)
        return self.fc(x)


In [6]:
model = PreTrainedClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # Move data to the device

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')

    return running_loss / len(train_loader)

# Validation function
def validate(model, test_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            # Forward pass
            output = model(data)

            # Calculate loss
            test_loss += criterion(output, target).item()

            # Get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'Average Test Loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)')
    return test_loss, accuracy


In [7]:
num_epochs = 10

for epoch in range(1, num_epochs + 1):
    print(f'Epoch {epoch}/{num_epochs}')

    # Train the model
    train_loss = train(model, train_loader, criterion, optimizer, device)

    # Validate the model
    test_loss, accuracy = validate(model, test_loader, criterion, device)

    print(f'Epoch {epoch} Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Accuracy: {accuracy:.2f}%')

# Save the trained model
torch.save(model.state_dict(), 'pre_trained_mnist_classifier.pth')


Epoch 1/10
Batch 0/938, Loss: 2.3180
Batch 100/938, Loss: 0.3620
Batch 200/938, Loss: 0.2353
Batch 300/938, Loss: 0.0997
Batch 400/938, Loss: 0.1727
Batch 500/938, Loss: 0.1763
Batch 600/938, Loss: 0.1232
Batch 700/938, Loss: 0.1176
Batch 800/938, Loss: 0.1509
Batch 900/938, Loss: 0.0506
Average Test Loss: 0.1145, Accuracy: 9644/10000 (96.44%)
Epoch 1 Train Loss: 0.2552 | Test Loss: 0.1145 | Accuracy: 96.44%
Epoch 2/10
Batch 0/938, Loss: 0.0758
Batch 100/938, Loss: 0.1046
Batch 200/938, Loss: 0.2735
Batch 300/938, Loss: 0.0488
Batch 400/938, Loss: 0.0840
Batch 500/938, Loss: 0.2048
Batch 600/938, Loss: 0.0359
Batch 700/938, Loss: 0.1151
Batch 800/938, Loss: 0.0594
Batch 900/938, Loss: 0.0334
Average Test Loss: 0.0972, Accuracy: 9697/10000 (96.97%)
Epoch 2 Train Loss: 0.1008 | Test Loss: 0.0972 | Accuracy: 96.97%
Epoch 3/10
Batch 0/938, Loss: 0.1219
Batch 100/938, Loss: 0.0550
Batch 200/938, Loss: 0.1024
Batch 300/938, Loss: 0.0241
Batch 400/938, Loss: 0.0078
Batch 500/938, Loss: 0.0252